<a href="https://colab.research.google.com/github/AdicherlaVenkataSai/H-CNNforfashionImageclassification/blob/master/VGG19HCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np
import pickle
import os
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from keras.initializers import he_normal
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.utils import to_categorical
from keras import backend as K

Using TensorFlow backend.


In [0]:
def scheduler(epoch):
  learning_rate_init = 0.001
  if epoch > 42:
    learning_rate_init = 0.0002
  if epoch > 52:
    learning_rate_init = 0.00005
  return learning_rate_init

In [0]:
def unpickle(filename):
  file = os.path.join(data_dir, filename)
  with open(file, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')
  return dict

In [0]:
class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    # customize your behavior
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 15:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 25:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 35:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)

In [0]:
height, width = 28, 28
channel = 1
if K.image_data_format() == 'channels_first':
    input_shape = (channel, height, width)
else:
    input_shape = (height, width, channel)

In [0]:
train_size = 60000
test_size = 10000

coarse1_classes = 2

coarse2_classes = 6

num_classes  = 10

batch_size   = 128
epochs       = 60

In [0]:
log_filepath = './tb_log_vgg19_hierarchy_dynamic/'
weights_store_filepath = './vgg19_weights_hierarchy_dynamic/'
retrain_id = '101'
model_name = 'weights_vgg16_fashionmnist'+retrain_id+'.h5'
model_path = os.path.join(weights_store_filepath, model_name)

In [8]:
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')

574717952/574710816 [==============================] - 53s 0us/step


In [9]:

(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
class_names_c1 = ['Clothes', 'Goods']
class_names_c2 = ['Tops', 'Bottoms', 'Dresses', 'Outers', 'Accessories', 'Shoes']

4423680/4422102 [==============================] - 2s 1us/step


In [0]:
c2_to_c1 = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1}
fine_to_c2 = {0:0, 1:1, 2:0, 3:2, 4:3, 5:5, 6:0, 7:5, 8:4, 9:5}

In [0]:
def print_mappings(mapping, source, dest):
    for k,v in mapping.items():
        print(source[k], "->", dest[v])

In [12]:
print_mappings(c2_to_c1, class_names_c2, class_names_c1)
print("-"*10)
print_mappings(fine_to_c2, class_names, class_names_c2)

Tops -> Clothes
Bottoms -> Clothes
Dresses -> Clothes
Outers -> Clothes
Accessories -> Goods
Shoes -> Goods
----------
T-shirt/top -> Tops
Trouser -> Bottoms
Pullover -> Tops
Dress -> Dresses
Coat -> Outers
Sandal -> Shoes
Shirt -> Tops
Sneaker -> Shoes
Bag -> Accessories
Ankle boot -> Shoes


In [13]:
train_images.shape

(60000, 28, 28)

In [14]:
train_labels_fine = to_categorical(train_labels)
train_labels_fine.shape

(60000, 10)

In [15]:
test_labels_fine = to_categorical(test_labels)
test_labels_fine.shape

(10000, 10)

In [16]:
train_labels_c2_index = [fine_to_c2[i] for i in train_labels]
train_labels_c2 = to_categorical(train_labels_c2_index)
train_labels_c2.shape

(60000, 6)

In [17]:
test_labels_c2_index = [fine_to_c2[i] for i in test_labels]
test_labels_c2 = to_categorical(test_labels_c2_index)
test_labels_c2.shape

(10000, 6)

In [18]:
train_labels_c1_index = [c2_to_c1[i] for i in train_labels_c2_index]
train_labels_c1 = to_categorical(train_labels_c1_index)
train_labels_c1.shape

(60000, 2)

In [19]:
test_labels_c1_index = [c2_to_c1[i] for i in test_labels_c2_index]
test_labels_c1 = to_categorical(test_labels_c1_index)
test_labels_c1.shape

(10000, 2)

In [0]:
x_train = train_images[..., np.newaxis]
x_test = test_images[..., np.newaxis]

y_train = train_labels_fine
y_test = test_labels_fine

y_c1_train = train_labels_c1
y_c1_test = test_labels_c1

y_c2_train = train_labels_c2
y_c2_test = test_labels_c2

In [21]:
print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)
print("y_c1_train shape: ", y_c1_train.shape)
print("y_c1_test shape: ", y_c1_test.shape)
print("y_c2_train shape: ", y_c2_train.shape)
print("y_c2_test shape: ", y_c2_test.shape)


x_train shape:  (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)
y_train shape:  (60000, 10)
y_test shape:  (10000, 10)
y_c1_train shape:  (60000, 2)
y_c1_test shape:  (10000, 2)
y_c2_train shape:  (60000, 6)
y_c2_test shape:  (10000, 6)


In [22]:
alpha = K.variable(value=0.98, dtype="float32", name="alpha") 
beta = K.variable(value=0.01, dtype="float32", name="beta") 
gamma = K.variable(value=0.01, dtype="float32", name="gamma") 

img_input = Input(shape=input_shape, name='input')
img_input

<tf.Tensor 'input:0' shape=(None, 28, 28, 1) dtype=float32>

In [0]:
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(coarse2_classes, activation='softmax')(c_2_bch)

#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax')(x)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='vgg19_hierarchy')


In [24]:
model.summary()

Model: "vgg19_hierarchy"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 64)   640         input[0][0]                      
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 64)   36928       batch_normalization_1[0][0]      
____________________________________________________________________________________

In [25]:
sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)
cbks = [change_lr, tb_cb, change_lw]

history = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

Train on 60000 samples, validate on 10000 samples
Epoch 1/60
60000/60000 [==============================] - 94s 2ms/step - loss: 0.1070 - dense_3_loss: 0.0652 - dense_6_loss: 1.5178 - dense_9_loss: 2.7964 - dense_3_accuracy: 0.9786 - dense_6_accuracy: 0.5157 - dense_9_accuracy: 0.2403 - val_loss: 0.0444 - val_dense_3_loss: 0.0251 - val_dense_6_loss: 0.5361 - val_dense_9_loss: 1.4303 - val_dense_3_accuracy: 0.9923 - val_dense_6_accuracy: 0.8141 - val_dense_9_accuracy: 0.5142
Epoch 2/60
60000/60000 [==============================] - 82s 1ms/step - loss: 0.0476 - dense_3_loss: 0.0265 - dense_6_loss: 0.7401 - dense_9_loss: 1.4183 - dense_3_accuracy: 0.9927 - dense_6_accuracy: 0.7511 - dense_9_accuracy: 0.5644 - val_loss: 0.0276 - val_dense_3_loss: 0.0160 - val_dense_6_loss: 0.3919 - val_dense_9_loss: 0.7841 - val_dense_3_accuracy: 0.9957 - val_dense_6_accuracy: 0.8638 - val_dense_9_accuracy: 0.7314
Epoch 3/60
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0370 - dens

In [26]:

'''model.compile(loss=keras.losses.categorical_crossentropy,
              # optimizer=keras.optimizers.Adadelta(),
              optimizer=sgd, 
              metrics=['accuracy'])'''

#model.save(model_path)
score = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
print('score is: ', score)

score is:  [0.42076459857193405, 0.008339118212461472, 0.11878246814012527, 0.42069384455680847, 0.9977999925613403, 0.9629999995231628, 0.9296000003814697]


In [0]:
# plot the loss and accuracy

import matplotlib.pyplot as plt
%matplotlib inline

loss = history.history['loss']
dense_3_loss = history.history['dense_3_loss']
dense_6_loss = history.history['dense_6_loss']
dense_9_loss = history.history['dense_9_loss']
dense_3_accuracy = history.history['dense_3_accuracy']
dense_6_accuracy = history.history['dense_6_accuracy']
dense_9_accuracy = history.history['dense_9_accuracy']
val_loss = history.history['val_loss']
val_dense_3_loss = history.history['val_dense_3_loss']
val_dense_6_loss = history.history['val_dense_6_loss']
val_dense_9_loss = history.history['val_dense_9_loss']
val_dense_3_accuracy = history.history['val_dense_3_accuracy']
val_dense_6_accuracy = history.history['val_dense_6_accuracy']
val_dense_9_accuracy = history.history['val_dense_9_accuracy']

epochs = range(1, 60)

plt.title('Training and validation accuracy')
plt.plot(epochs, dense_3_accuracy, 'red', label='Training C1 accuracy')
plt.plot(epochs, dense_6_accuracy, 'blue', label='Training C2 accuracy')
plt.plot(epochs, dense_9_accuracy, 'green', label='Training F accuracy')
plt.plot(epochs, val_dense_3_accuracy, 'yellow', label='Validation C1 accuracy')
plt.plot(epochs, val_dense_6_accuracy, 'violet', label='Validation C2 accuracy')
plt.plot(epochs, val_dense_9_accuracy, 'gray', label='Validation F accuracy')
plt.legend()

plt.figure()
plt.title('Training and validation loss')
plt.plot(epochs, dense_3_loss, 'red', label='Training C1 loss')
plt.plot(epochs, dense_6_loss, 'blue', label='Training C2 loss')
plt.plot(epochs, dense_9_loss, 'green', label='Training F loss')
plt.plot(epochs, val_dense_3_loss, 'yellow', label='Validation C1 loss')
plt.plot(epochs, val_dense_6_loss, 'violet', label='Validation C2 loss')
plt.plot(epochs, val_dense_9_loss, 'gray', label='Validation F loss')

plt.legend()

plt.show()